# Segmentación Semántica con UNet

En este notebook aprenderás a utilizar el modelo de segmentación semántica U-Net.

In [1]:
# Clonamos el repositorio a la carpeta 'unet'
!git clone -n https://github.com/milesial/Pytorch-UNet.git unet

"git" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [2]:
# Nos posicionamos en la carpeta 'unet'
%cd unet

[WinError 2] El sistema no puede encontrar el archivo especificado: 'unet'
d:\Alvaro\Cursos\PUCP\M2\Desarrollo en Aplicaciones con Visión Artificial\TrabajoFinal\Modelo\Covid-19\COVID-19_Radiography_Dataset


C:\Users\Sistema\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\magics\osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [3]:
!git checkout e1a69e7c6ce18edd47271b01e4aabc03b436753d

"git" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


### Ejemplo de Predicción

In [5]:
!pip install utils

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13934 sha256=d099466d60a57974d2dd364212494e154625db8fd6895dc5b9fa5da35d5e4351
  Stored in directory: c:\users\sistema\appdata\local\packages\pythonsoftwarefoundation.python.3.12_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\b6\a1\81\1036477786ae0e17b522f6f5a838f9bc4288d1016fc5d0e1ec
Successfully built utils


In [13]:
import sys
print(sys.path)


['d:\\Alvaro\\Cursos\\PUCP\\M2\\Desarrollo en Aplicaciones con Visión Artificial\\TrabajoFinal\\Modelo\\Covid-19\\COVID-19_Radiography_Dataset', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\\python39.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\\lib', 'C:\\Users\\Sistema\\AppData\\Local\\Microsoft\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0', '', 'C:\\Users\\Sistema\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python39\\site-packages', 'C:\\Users\\Sistema\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python39\\site-packages\\win32', 'C:\\Users\\Sistema\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\\L

In [14]:
# Importamos librerias necesarias
import torch
from unet import *
from PIL import Image
from utils.data_loading import BasicDataset
from torchvision import transforms
import torch.nn.functional as F
import numpy as np
import plotly.express as px

ModuleNotFoundError: No module named 'utils.data_loading'

In [ ]:
# Descargamos el modelo pre-entrenado y lo guardamos como 'MODEL.pth'
!wget -O MODEL.pth https://github.com/milesial/Pytorch-UNet/releases/download/v2.0/unet_carvana_scale0.5_epoch1.pth

In [ ]:
# Definimos el modelo y cargamos los pesos pre-entrenados
net = UNet(n_channels=3, n_classes=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(device)
net.load_state_dict(torch.load("MODEL.pth", map_location=device))
print(net)

In [ ]:
# Definimos algunas funciones
def predict_img(net,
                full_img,
                device,
                scale_factor=1,
                out_threshold=0.5):
    ''' transformando imagen '''
    net.eval()

    img = torch.from_numpy(BasicDataset.preprocess(full_img, scale_factor, is_mask=False))
    img = img.unsqueeze(0)
    img = img.to(device=device, dtype=torch.float32)

    with torch.no_grad():
        output = net(img)

        if net.n_classes > 1:
            probs = F.softmax(output, dim=1)[0]
        else:
            probs = torch.sigmoid(output)[0]

        tf = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((full_img.size[1], full_img.size[0])),
            transforms.ToTensor()
        ])

        full_mask = tf(probs.cpu()).squeeze()

    if net.n_classes == 1:
        return (full_mask > out_threshold).numpy()
    else:
        return F.one_hot(full_mask.argmax(dim=0), net.n_classes).permute(2, 0, 1).numpy()


def mask_to_image(mask):
    if mask.ndim == 2:
        return Image.fromarray((mask * 255).astype(np.uint8))
    elif mask.ndim == 3:
        return Image.fromarray((np.argmax(mask, axis=0) * 255 / mask.shape[0]).astype(np.uint8))


In [ ]:
# Descargmos una imagen de prueba
!wget https://github.com/diplomado-ia-pucp/dava/raw/main/car2.jpg

In [ ]:
# Predicción
img = Image.open('car2.jpg')

mask = predict_img(net=net,
                    full_img=img,
                    scale_factor=0.5,
                    out_threshold=0.5,
                    device=device)

result = mask_to_image(mask)

px.imshow(np.asarray(img)).show()

px.imshow(np.asarray(result), binary_string=True).show()